
# 🦙Llama-cpp users🦙
## If you are just using llama-cpp then follow these steps

The a folder containing the bin file should be located in the models folder 

Example: "./models/Alpaca-7B-ggml-4bit-LoRA-merged/ggml-model-q4_0.bin"
## Install and Import dependencies

In [2]:
!pip install llama-cpp-python
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.7 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.34-cp310-cp310-linux_x86_64.whl size=138481 sha256=b7bf19c821ef39c0b6ba48494d59613ddc0927cd83d92a7afe1492db10fab30d
  Stored in directory: /home/captdishwasher/.cache/pip/wheels/c3/df/54/e25ec8fe359db8240b51f3849e8b8de07ffc693f880ebf40c4
Successfully built llama-cpp-python


In [3]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain

# Select Model

In [25]:
model_dir = "/media/captdishwasher/Samshmung/horenbergerb/llama/llama.cpp/models/"
import os
# get a list of all folders in the models directory
model_folders = [f for f in os.listdir(model_dir) if os.path.isdir(os.path.join(model_dir, f))]

# print the list of model names with their index starting at 1
for i, model_name in enumerate(model_folders):
    print(f"{i+1}. {model_name}")

# ask the user to select a model by number
selected_index = int(input("Enter the number of the model to select: ")) - 1
selected_model = model_folders[selected_index]

# check if the selected model contains a .bin file and save the path if it does
model_bin = None
for file in os.listdir(os.path.join(model_dir, selected_model)):
    if file.endswith(".bin"):
        model_bin = os.path.join(model_dir, selected_model, file)
        break

if model_bin:
    print(f"Selected model binary: {model_bin}")
else:
    print("No .bin file found in selected model directory.")
    
llm = LlamaCpp(model_path=model_bin, n_ctx=2048, n_threads=7, temperature=0.65)

1. toolpaca-13b-native-ggml-model-q4_0
2. Alpacino-13b-ggml
3. llama-13b-pretrained-sft-do2-ggml-q4
4. llama-13b-pretrained-dropout-ggml-q4_0
5. gpt4all-j
6. Alpaca13B
7. vicuna-13B-1.1-GPTQ-4bit-128g-GGML
8. llama-30b
9. vicuna-13b-ggml-q4_0-delta-merged
10. oasst-llama30b-ggml-q4
11. GPT4-x-Alpaca-13B
12. gpt4-alpaca-lora-30B-4bit-GGML
13. llama-13b-ggml-q4_0
Selected model binary: /media/captdishwasher/Samshmung/horenbergerb/llama/llama.cpp/models/vicuna-13B-1.1-GPTQ-4bit-128g-GGML/vicuna-13B-1.1-GPTQ-4bit-128g.GGML.bin


llama.cpp: loading model from /media/captdishwasher/Samshmung/horenbergerb/llama/llama.cpp/models/vicuna-13B-1.1-GPTQ-4bit-128g-GGML/vicuna-13B-1.1-GPTQ-4bit-128g.GGML.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 4 (mostly Q4_1, some F16)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =  73.73 KB
llama_model_load_internal: mem required  = 11749.65 MB (+ 3216.00 MB per state)
llama_init_from_file: kv self size  = 3200.00 MB
AVX = 1 | AVX2 = 1 | AVX512 = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | F

# Begginning of Langchain section
I stole some of the code from [this colab](https://colab.research.google.com/drive/1VOwJpcZqOXag-ZXi-52ibOx6L5Pw-YJi#scrollTo=nu-AmhDLEK0h) that goes with [this video](https://www.youtube.com/watch?v=LbT1yp6quS8) by Patrick Loeber. I recommend subscribing.

In [37]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, GoogleSearchAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

In [38]:
# Define which tools the agent can use to answer user queries
search = GoogleSearchAPIWrapper(google_api_key='DUMMYKEY', google_cse_id='DUMMYKEY')
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [67]:
# Set up the base template
template = """User: Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Answer using exactly the following format:

```
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
```

Your answers should always begin with "Question:".

Assistant: Understood. I will answer further questions using exactly the provided format. My final answer will be formatted in pirate speak.
User: {input}
Assistant: {agent_scratchpad}"""

In [68]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [69]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [70]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [71]:
output_parser = CustomOutputParser()

In [72]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [73]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [74]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [75]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
 Question: How many people live in Canada as of 2023?
Thought: I need to know the current population of Canada.
Action: Search
Action Input: "Canadian population 2023"


llama_print_timings:        load time =  2996.85 ms
llama_print_timings:      sample time =    19.16 ms /    54 runs   (    0.35 ms per run)
llama_print_timings: prompt eval time = 76356.40 ms /   220 tokens (  347.07 ms per token)
llama_print_timings:        eval time = 28620.48 ms /    53 runs   (  540.01 ms per run)
llama_print_timings:       total time = 105000.47 ms




Observation:The current population of Canada is 38,662,830 as of Monday, April 17, 2023, based on Worldometer elaboration of the latest United Nations data. · Canada 2020 ... Mar 22, 2023 ... Record-high population growth in the year 2022. Canada's population was estimated at 39,566,248 on January 1, 2023, after a record population ... Mar 22, 2023 ... Total population grew by a record 1.05 million people to 39.57 million in the twelve months to Jan. 1, 2023, and about 96% of the rise was due ... 14 records ... Estimated number of persons by quarter of a year and by year, Canada, provinces and ... Geography, Q1 2022, Q2 2022, Q3 2022, Q4 2022, Q1 2023 ... Mar 22, 2023 ... The North American country documented an increase of more than 1 million citizens in 2022, bringing its population to more than 39.5 million, ... The current population of Canada in 2023 is 38,781,291, a 0.85% increase from 2022. The population of Canada in 2022 was 38,454,327, a 0.78% increase from 2021. Mar 17, 20


llama_print_timings:        load time =  2996.85 ms
llama_print_timings:      sample time =    29.56 ms /    82 runs   (    0.36 ms per run)
llama_print_timings: prompt eval time = 300777.62 ms /   853 tokens (  352.61 ms per token)
llama_print_timings:        eval time = 48956.60 ms /    81 runs   (  604.40 ms per run)
llama_print_timings:       total time = 349773.68 ms


"Arrrr, 38,662,830 be the current population o' Canada as o' 2023, with a wee bit o' increase from the previous year."